# Segmenting Neighborhoods in Toronto

In this project, we will describe the different geographical subdivisions of the city of Toronto, Canada.

### Import Relevant Libraries

In [1]:
import pandas as pd
import numpy as np
import requests

import numpy as np # library to handle data in a vectorized manner
import json # library to handle JSON files
import matplotlib.cm as cm # Matplotlib and associated plotting modules
import matplotlib.colors as colors

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from pandas.io.html import read_html
from sklearn.cluster import KMeans # import k-means from clustering stage

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print('Libraries imported.')


Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.19.0-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  23.65 MB/s
geopy-1.19.0-p 100% |################################| Time: 0:00:00  34.58 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  52.47 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  35.11 MB/s
vincent-0.4.4- 100% |###################

### Scrape the Data from Wikipedia

First, scrape data from Wikipedia page and create a table. 

In [4]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [5]:
#print(table.prettify()) # this helps us see how the tags are nested.

In [6]:
wikitables = read_html(url, attrs={"class":"wikitable"})
print("Extracted {num} wikitables".format(num=len(wikitables)))

Extracted 1 wikitables


In [7]:
wikitables # only run this if you want to check what the wikitable looks like.

[            0                 1  \
 0    Postcode           Borough   
 1         M1A      Not assigned   
 2         M2A      Not assigned   
 3         M3A        North York   
 4         M4A        North York   
 5         M5A  Downtown Toronto   
 6         M5A  Downtown Toronto   
 7         M6A        North York   
 8         M6A        North York   
 9         M7A      Queen's Park   
 10        M8A      Not assigned   
 11        M9A         Etobicoke   
 12        M1B       Scarborough   
 13        M1B       Scarborough   
 14        M2B      Not assigned   
 15        M3B        North York   
 16        M4B         East York   
 17        M4B         East York   
 18        M5B  Downtown Toronto   
 19        M5B  Downtown Toronto   
 20        M6B        North York   
 21        M7B      Not assigned   
 22        M8B      Not assigned   
 23        M9B         Etobicoke   
 24        M9B         Etobicoke   
 25        M9B         Etobicoke   
 26        M9B         Etobi

### Extract the values from the table into headers and row data

Next, we will extract the name of the headers from the wikitable.

In [8]:
postcode_hdr = wikitables[0][0][0]
borough_hdr = wikitables[0][1][0]
neighborhood_hdr = wikitables[0][2][0]


Let's extract the values from the wikitable for each data type and and them to a list.


In [9]:
postcodes = []
boroughs = []
neighborhoods = []

#check size of wikitables and change for loop range to automate range. 
#Consider adding a space for entries that come after the first one in the list.

for entry in range(1,289):
    borough = wikitables[0][1][entry]
    neighborhood = wikitables[0][2][entry]
    postcode = wikitables[0][0][entry]
    if borough != 'Not assigned':
        boroughs.append(borough)
        if neighborhood != 'Not assigned':
            neighborhoods.append(neighborhood)
        else:
            neighborhoods.append(borough)
        postcodes.append(postcode)

Now let's add each list to a Pandas dataframe and display the dataframe.

In [10]:
df = pd.DataFrame({postcode_hdr:postcodes,borough_hdr:boroughs,neighborhood_hdr:neighborhoods})

In [11]:
df = df[['Postcode','Borough','Neighbourhood']] # this will display the table with the required column order
df

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


We should check how many distinct postcodes are in the dataset before we do any grouping. This number should equal the number of entries in the resulting dataframe after we group.

In [12]:
uniquePostcodes = df.Postcode.unique()

size = uniquePostcodes.size
size

103

### Grouping Data

In the next cell, we are grouping Neighborhoods that are in the same Postcode. 
We also drop the duplicate Postcodes, assuming all our data is right.

In [13]:
df['Neighbourhood'] = df[['Borough','Neighbourhood','Postcode']].groupby(['Postcode'])['Neighbourhood'].transform(lambda x: ','.join(x))
df = df[['Borough','Neighbourhood','Postcode']].drop_duplicates(['Postcode'])
df


,Borough,Neighbourhood,Postcode
0,North York,Parkwoods,M3A
1,North York,Victoria Village,M4A
2,Downtown Toronto,"Harbourfront,Regent Park",M5A
4,North York,"Lawrence Heights,Lawrence Manor",M6A
6,Queen's Park,Queen's Park,M7A
7,Etobicoke,Islington Avenue,M9A
8,Scarborough,"Rouge,Malvern",M1B
10,North York,Don Mills North,M3B
11,East York,"Woodbine Gardens,Parkview Hill",M4B
13,Downtown Toronto,"Ryerson,Garden District",M5B


### Clean Up Dataframe

Now, let's reset the index values.

In [14]:
df = df.reset_index()

In [15]:
df = df.drop('index', axis = 1)

Clean up the dataframe (display columns in the desired order)

In [16]:
df = df[['Postcode', 'Borough','Neighbourhood']]
df

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


Let's check that the number of entries in the data frame equals 103.

In [17]:
df.shape

(103, 3)

## This looks great!

# Now, let's find Latitudes and Longitudes for each Postal Code.

In [18]:
#Hey, I really tried to do it this way!
#latitude = []
#longitude = []
#For each postcode in the dataframe, get the postcode, and look for the lat and long.

#for index in range(0,102):
#    lat_lng_coords = None
#    bor = df.iloc[index][1] + ', Ontario, Canada, ' + df_copy.iloc[index][0]
#    print(bor)
    
    #while(lat_lng_coords is None):  # loop until you get the coordinates
     #   with requests.Session() as session:
      #      g = geocoder.google(bor, session=session)
       #     g = geocoder.google(bor)
      #      print(g)
      #      lat_lng_coords = g.latlng
       #     print(lat_lng_coords)
    #latitude.append(lat_lng_coords[0])
    #longitude.append(lat_lng_coords[1])
    

In [20]:
latlong_df = pd.read_csv('http://cocl.us/Geospatial_data')
latlong_df

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


We want to sort the datasets in a similar way (by Postal Code)

In [21]:
latlong_df_sort = latlong_df.sort_values(by='Postal Code')
df_sort = df.sort_values(by='Postcode')

Now, we have to check that the Postal Codes in the CSV and in the original dataframe match exactly.

In [22]:
check =[]
for index in range(0,103):
    a = df_sort.iloc[index][0]
    b = latlong_df_sort.iloc[index][0]
    if a != b:
        print('Postcodes in Datasets Do Not Match')
print('Finished checking!')

Finished checking!


GREAT, THEY MATCH! 
GOOD, BECAUSE EVERYBODY PROBABLY WASTED MANY HOURS TRYING TO MAKE GEOCODER WORK...

It's easy now to add the Lat and Lon data to our original dataframe.

In [23]:
df_sort['Latitude'] = latlong_df_sort['Latitude']
df_sort['Longitude'] = latlong_df_sort['Longitude']

In [24]:
df_sort = df_sort.reset_index()


In [25]:
df_sort = df_sort.drop('index', axis = 1)

In [26]:
df_sort

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.727929,-79.262029
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.794200,-79.262029
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.778517,-79.346556
3,M1G,Scarborough,Woburn,43.770120,-79.408493
4,M1H,Scarborough,Cedarbrae,43.745906,-79.352188
5,M1J,Scarborough,Scarborough Village,43.728496,-79.495697
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.709060,-79.363452
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.728020,-79.388790
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.667967,-79.367675
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.650571,-79.384568


In [220]:
from geopy import Nominatim
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


## Make a Map of Toronto and All Its Venues Extracted from FourSquare

In [221]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_sort['Latitude'], df_sort['Longitude'], df_sort['Borough'], df_sort['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Foursquare Credentials

In [222]:
CLIENT_ID = 'HJQTB2PO3CQ31PY0D3MKAFCODL1XOO2RLY3VXWZ2XVOUHERI' # your Foursquare ID
CLIENT_SECRET = 'YVXU2GICCUXXHV00HDZUG2ZCR5WG50VYWQCCF14A5JJYY31Y' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HJQTB2PO3CQ31PY0D3MKAFCODL1XOO2RLY3VXWZ2XVOUHERI
CLIENT_SECRET:YVXU2GICCUXXHV00HDZUG2ZCR5WG50VYWQCCF14A5JJYY31Y


## define a function to extract nearby venues for each neighborhood

In [334]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
 
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## Create the dataframe storing the nearby venues

In [335]:
toronto_venues = getNearbyVenues(names=df_sort['Neighbourhood'],
                                   latitudes=df_sort['Latitude'],
                                   longitudes=df_sort['Longitude']
                                  )

In [337]:
print(toronto_venues.shape)
toronto_venues.head()

(2245, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.727929,-79.262029,Giant Tiger,43.727447,-79.266240,Department Store
1,"Rouge,Malvern",43.727929,-79.262029,Tim Hortons,43.726895,-79.266157,Coffee Shop
2,"Rouge,Malvern",43.727929,-79.262029,Hakka No.1,43.727688,-79.266057,Chinese Restaurant
3,"Rouge,Malvern",43.727929,-79.262029,Dollarama,43.726904,-79.265886,Discount Store
4,"Rouge,Malvern",43.727929,-79.262029,Tandy Leather,43.726974,-79.266513,Hobby Shop


In [338]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",3,3,3,3,3,3
Agincourt,20,20,20,20,20,20
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",39,39,39,39,39,39
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",10,10,10,10,10,10
"Bathurst Manor,Downsview North,Wilson Heights",18,18,18,18,18,18
Bayview Village,17,17,17,17,17,17
"Bedford Park,Lawrence Manor East",100,100,100,100,100,100
Berczy Park,2,2,2,2,2,2
"Birch Cliff,Cliffside West",100,100,100,100,100,100


In [339]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 275 uniques categories.


## Create One Hot Encoded cells for each venue type

In [340]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [341]:
toronto_onehot.shape

(2245, 275)

In [342]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"Adelaide,King,Richmond",0.000000,0.000000,0.000000,0.00

In [343]:
toronto_grouped.shape

(100, 275)

In [344]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
           venue  freq
0       Bus Stop  0.33
1        Airport  0.33
2           Park  0.33
3    Yoga Studio  0.00
4  Metro Station  0.00


----Agincourt----
                    venue  freq
0                    Café  0.10
1             Coffee Shop  0.10
2          Breakfast Spot  0.10
3  Furniture / Home Store  0.05
4    Caribbean Restaurant  0.05


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                 venue  freq
0          Coffee Shop  0.23
1  Japanese Restaurant  0.05
2         Burger Joint  0.05
3                Diner  0.05
4                  Gym  0.05


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
            venue  freq
0     Pizza Place   0.2
1     Coffee Shop   0.1
2    Skating Rink   0.1
3  Sandwich Place   0.1
4             Pub   0.1


----Bathurst Manor,Downsview North,Wilson Heights----
                 venue  freq
0          Coffee Shop  0.11
1         

                 venue  freq
0          Coffee Shop  0.10
1  Sporting Goods Shop  0.10
2         Burger Joint  0.06
3    Electronics Store  0.03
4           Beer Store  0.03


----East Toronto----
                  venue  freq
0           Pizza Place  0.15
1  Fast Food Restaurant  0.15
2              Pharmacy  0.08
3  Gym / Fitness Center  0.08
4    Rock Climbing Spot  0.08


----Emery,Humberlea----
                venue  freq
0         Coffee Shop  0.15
1  Italian Restaurant  0.05
2        Burger Joint  0.04
3                Café  0.04
4      Sandwich Place  0.04


----Fairview,Henry Farm,Oriole----
                  venue  freq
0    Athletics & Sports  0.25
1  Gym / Fitness Center  0.25
2           Coffee Shop  0.25
3         Grocery Store  0.25
4         Metro Station  0.00


----First Canadian Place,Underground city----
                        venue  freq
0              Baseball Field   0.5
1      Furniture / Home Store   0.5
2                 Men's Store   0.0
3  Modern European R

                   venue  freq
0                   Café  0.25
1        College Stadium  0.25
2           Skating Rink  0.25
3  General Entertainment  0.25
4     Mexican Restaurant  0.00


----Scarborough Village----
               venue  freq
0     Baseball Field  0.33
1         Food Truck  0.33
2   Business Service  0.33
3  Martial Arts Dojo  0.00
4     Massage Studio  0.00


----Silver Hills,York Mills----
              venue  freq
0       Pizza Place  0.09
1    Clothing Store  0.09
2    Sandwich Place  0.09
3  Asian Restaurant  0.09
4      Dance Studio  0.09


----St. James Town----
                    venue  freq
0    Fast Food Restaurant  0.17
1      Chinese Restaurant  0.17
2  Thrift / Vintage Store  0.08
3             Pizza Place  0.08
4             Coffee Shop  0.08


----Stn A PO Boxes 25 The Esplanade----
                    venue  freq
0          Discount Store  0.07
1  Thrift / Vintage Store  0.07
2             Social Club  0.07
3            Burger Joint  0.07
4            

In [345]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)

    return row_categories_sorted.index.values[0:num_top_venues]

# Clustering Exercise

## For this part, I decided to analyse 6 types of venues  in Toronto, after a bit of trial and error. 

In [347]:
num_top_venues = 6

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue
0,"Adelaide,King,Richmond",Park,Airport,Bus Stop,Electronics Store,Eastern European Restaurant,Empanada Restaurant
1,Agincourt,Coffee Shop,Café,Breakfast Spot,Performing Arts Venue,Bar,Gym
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Coffee Shop,Diner,Gym,Burger Joint,Japanese Restaurant,Yoga Studio
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Pizza Place,Coffee Shop,Gym,Dance Studio,Skating Rink,Pharmacy
4,"Bathurst Manor,Downsview North,Wilson Heights",Coffee Shop,Frozen Yogurt Shop,Diner,Sandwich Place,Bridal Shop,Fast Food Restaurant


## Only 3 clusters were found to be significant in the whole of Toronto.

In [348]:
# set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10].astype(int)

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [349]:
df_sort

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.727929,-79.262029
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.794200,-79.262029
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.778517,-79.346556
3,M1G,Scarborough,Woburn,43.770120,-79.408493
4,M1H,Scarborough,Cedarbrae,43.745906,-79.352188
5,M1J,Scarborough,Scarborough Village,43.728496,-79.495697
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.709060,-79.363452
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.728020,-79.388790
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.667967,-79.367675
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.650571,-79.384568


In [350]:
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue
0,"Adelaide,King,Richmond",Park,Airport,Bus Stop,Electronics Store,Eastern European Restaurant,Empanada Restaurant
1,Agincourt,Coffee Shop,Café,Breakfast Spot,Performing Arts Venue,Bar,Gym
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Coffee Shop,Diner,Gym,Burger Joint,Japanese Restaurant,Yoga Studio
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Pizza Place,Coffee Shop,Gym,Dance Studio,Skating Rink,Pharmacy
4,"Bathurst Manor,Downsview North,Wilson Heights",Coffee Shop,Frozen Yogurt Shop,Diner,Sandwich Place,Bridal Shop,Fast Food Restaurant
5,Bayview Village,Indian Restaurant,Yoga Studio,Supermarket,Grocery Store,Gym,Intersection
6,"Bedford Park,Lawrence Manor East",Coffee Shop,Restaurant,Café,Hotel,Breakfast Spot,Gastropub
7,Berczy Park,Cafeteria,Martial Arts Dojo,Women's Store,Dog Run,Department Store,Dessert Shop
8,"Birch Cliff,Cliffside West",Coffee Shop,Bar,Café,Thai Restaurant,Steakhouse,Hotel
9,"Bloordale Gardens,Eringate,Markland Wood,Old B...",Pool,Mediterranean Restaurant,Golf Course,Athletics & Sports,Dog Run,Deli / Bodega


In [351]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'ClusterLabels', kmeans.labels_.astype(int))

toronto_merged = df_sort

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.727929,-79.262029,1.0,Discount Store,Coffee Shop,Chinese Restaurant,Department Store,Hobby Shop,Bus Station
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.794200,-79.262029,1.0,Lounge,Clothing Store,Sandwich Place,Breakfast Spot,Doner Restaurant,Dessert Shop
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.778517,-79.346556,1.0,Clothing Store,Fast Food Restaurant,Coffee Shop,Bus Station,Toy / Game Store,Restaurant
3,M1G,Scarborough,Woburn,43.770120,-79.408493,1.0,Coffee Shop,Ramen Restaurant,Restaurant,Sandwich Place,Café,Pizza Place
4,M1H,Scarborough,Cedarbrae,43.745906,-79.352188,1.0,Gym / Fitness Center,Caribbean Restaurant,Café,Japanese Restaurant,Basketball Court,Baseball Field
5,M1J,Scarborough,Scarborough Village,43.728496,-79.495697,1.0,Business Service,Food Truck,Baseball Field,Doner Restaurant,Dessert Shop,Dim Sum Restaurant
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.709060,-79.363452,1.0,Coffee Shop,Sporting Goods Shop,Burger Joint,Sushi Restaurant,Restaurant,Bagel Shop
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.728020,-79.388790,0.0,Park,Bus Line,Swim School,Discount Store,Department Store,Dessert Shop
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.667967,-79.367675,1.0,Coffee Shop,Restaurant,Bakery,Italian Restaurant,Pub,Café
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.650571,-79.384568,1.0,Coffee Shop,Bar,Café,Thai Restaurant,Steakhouse,Hotel


In [352]:
toronto_merged = toronto_merged.dropna()

In [327]:
toronto_merged

,Postcode,Borough,Neighbourhood,Latitude,Longitude,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.727929,-79.262029,1.0,Discount Store,Coffee Shop,Chinese Restaurant,Department Store,Hobby Shop,Bus Station
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.794200,-79.262029,1.0,Lounge,Clothing Store,Sandwich Place,Breakfast Spot,Doner Restaurant,Dessert Shop
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.778517,-79.346556,1.0,Clothing Store,Fast Food Restaurant,Coffee Shop,Bus Station,Toy / Game Store,Restaurant
3,M1G,Scarborough,Woburn,43.770120,-79.408493,1.0,Coffee Shop,Ramen Restaurant,Restaurant,Sandwich Place,Café,Pizza Place
4,M1H,Scarborough,Cedarbrae,43.745906,-79.352188,1.0,Gym / Fitness Center,Caribbean Restaurant,Café,Japanese Restaurant,Basketball Court,Baseball Field
5,M1J,Scarborough,Scarborough Village,43.728496,-79.495697,1.0,Business Service,Food Truck,Baseball Field,Doner Restaurant,Dessert Shop,Dim Sum Restaurant
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.709060,-79.363452,1.0,Coffee Shop,Sporting Goods Shop,Burger Joint,Sushi Restaurant,Restaurant,Bagel Shop
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.728020,-79.388790,0.0,Park,Bus Line,Swim School,Discount Store,Department Store,Dessert Shop
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.667967,-79.367675,1.0,Coffee Shop,Restaurant,Bakery,Italian Restaurant,Pub,Café
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.650571,-79.384568,1.0,Coffee Shop,Bar,Café,Thai Restaurant,Steakhouse,Hotel


In [328]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['ClusterLabels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Analyse Clusters

## Cluster 1: Near Parks and Trails and Family Friendly locations

In [329]:
toronto_merged.loc[toronto_merged['ClusterLabels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue
7,Scarborough,0.0,Park,Bus Line,Swim School,Discount Store,Department Store,Dessert Shop
15,Scarborough,0.0,Park,River,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore
24,North York,0.0,Park,Japanese Restaurant,Bakery,Pub,Dance Studio,Department Store
30,North York,0.0,Park,Coffee Shop,Convenience Store,Dog Run,Department Store,Dessert Shop
36,East York,0.0,Park,Playground,Dog Run,Deli / Bodega,Department Store,Dessert Shop
47,Central Toronto,0.0,Park,Basketball Court,Bakery,Construction & Landscaping,Eastern European Restaurant,Electronics Store
58,Downtown Toronto,0.0,Park,Airport,Bus Stop,Electronics Store,Eastern European Restaurant,Empanada Restaurant
65,Central Toronto,0.0,Park,Women's Store,Fast Food Restaurant,Market,Pharmacy,Gift Shop
75,Downtown Toronto,0.0,Park,Fast Food Restaurant,Food & Drink Shop,Bus Stop,Women's Store,Diner
87,East Toronto,0.0,Park,Pizza Place,Bus Line,Mobile Phone Shop,Dumpling Restaurant,Drugstore


## Cluster 2: Near A Coffee Shop, Restaurants, and Bars

In [332]:
toronto_merged.loc[toronto_merged['ClusterLabels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue
0,Scarborough,1.0,Discount Store,Coffee Shop,Chinese Restaurant,Department Store,Hobby Shop,Bus Station
1,Scarborough,1.0,Lounge,Clothing Store,Sandwich Place,Breakfast Spot,Doner Restaurant,Dessert Shop
2,Scarborough,1.0,Clothing Store,Fast Food Restaurant,Coffee Shop,Bus Station,Toy / Game Store,Restaurant
3,Scarborough,1.0,Coffee Shop,Ramen Restaurant,Restaurant,Sandwich Place,Café,Pizza Place
4,Scarborough,1.0,Gym / Fitness Center,Caribbean Restaurant,Café,Japanese Restaurant,Basketball Court,Baseball Field
5,Scarborough,1.0,Business Service,Food Truck,Baseball Field,Doner Restaurant,Dessert Shop,Dim Sum Restaurant
6,Scarborough,1.0,Coffee Shop,Sporting Goods Shop,Burger Joint,Sushi Restaurant,Restaurant,Bagel Shop
8,Scarborough,1.0,Coffee Shop,Restaurant,Bakery,Italian Restaurant,Pub,Café
9,Scarborough,1.0,Coffee Shop,Bar,Café,Thai Restaurant,Steakhouse,Hotel
10,Scarborough,1.0,Coffee Shop,Sandwich Place,Café,Pizza Place,Park,Cheese Shop


## Cluster 3: Near A Bank

In [333]:
toronto_merged.loc[toronto_merged['ClusterLabels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue
38,East York,2.0,Bank,Park,Women's Store,Department Store,Dessert Shop,Dim Sum Restaurant
102,Etobicoke,2.0,Bank,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
